In [1]:
import cv2
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Function
from torchvision import models, transforms
from make_dloader import make_data
from torch.utils.data import DataLoader
from utils import bbox_collate, MixedRandomSampler
import transform as transf
import yaml
import json
from matplotlib import pyplot as plt
import numbers
import torchvision
import torch.optim as optim
import imgaug as ia
import imgaug.augmenters as iaa
from imgaug import parameters as iap
import copy
from model import ResNet50
import time
from scipy.ndimage import gaussian_filter
import random

config = yaml.safe_load(open('./config.yaml'))
dataset_means = json.load(open(config['dataset']['mean_file']))

In [2]:
class FeatureExtractor():
    """ Class for extracting activations and 
    registering gradients from targetted intermediate layers """

    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x

In [3]:
class ModelOutputs():

    def __init__(self, model, feature_module, target_layers):
        self.model = model
        self.feature_module = feature_module
        self.feature_extractor = FeatureExtractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)
            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
            else:
                x = module(x)
        
        return target_activations, x


def cam_on_image(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * (1-mask)), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)/255
    cam = cam / np.max(cam)
    return cam
   
def thresh(narray, threshold = 0.15, binary = False):
    if binary:
        return np.where(narray>threshold*np.max(narray), 1, 0)
    return np.where(narray>threshold*np.max(narray), narray, 0)

In [4]:
class GradCam:
    def __init__(self, model, feature_module, target_layer_names, use_cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = use_cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = ModelOutputs(self.model, self.feature_module, target_layer_names)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        if self.cuda:
            features, output = self.extractor(input.cuda())
        else:
            features, output = self.extractor(input)
        o = torch.sigmoid(output)
        o = o.cpu().data.numpy()
        if index == None:
            o = np.where(o>0.5, 1., 0.)
        label = o.sum(axis = 0)
        #print(label)
        label = np.where(label>o.shape[0]/2, 1., 0.)
        cam_list = []
        for i in range(len(label)):
            if label[i] == 0:
                cam_list.append(0)
                continue
                
            one_hot = np.zeros_like(label)
            one_hot[i] = 1.
            one_hot = torch.from_numpy(one_hot).requires_grad_(True)
            if self.cuda:
                one_hot = torch.sum(one_hot.cuda() * output)
            else:
                one_hot = torch.sum(one_hot * output)
            
            self.feature_module.zero_grad()
            self.model.zero_grad()
            one_hot.backward(retain_graph=True)

            grads_val = self.extractor.get_gradients()[-1].cpu().data.numpy()

            target = features[-1]
            target = target.cpu().data.numpy() # 40,2048,16,16

            weights = np.mean(grads_val, axis=(2, 3)) #40,2048
            weights = weights[:,:,np.newaxis,np.newaxis] #40,2048,1,1
            cam = np.zeros((target.shape[0], target.shape[2], target.shape[3]), dtype=np.float32) #40,16,16
            target =  weights * target #40,2048,16,16
            target = target.sum(axis=1) #40,16,16
            target = np.maximum(target, 0)
            T = np.zeros((input.shape[0],input.shape[2],input.shape[3]))
            for b in range(input.shape[0]):
                cam = cv2.resize(target[b], input.shape[2:])
                cam = cam - np.min(cam)
                cam = cam / np.max(cam)
                T[b] = cam
            cam_list.append(T)
            
        #if input.shape[0] == 1:
        #    return cam_list
        return label, cam_list


In [6]:
#model = models.resnet50(pretrained=False)
#model.fc = nn.Sequential( nn.Linear(in_features=2048, out_features=4, bias=True) )
model = ResNet50()
model.cuda()
model.load_state_dict(torch.load("/data/unagi0/masaoka/resnet50_classify0.pt"))

<All keys matched successfully>

In [ ]:
grad_cam = GradCam(model=model.resnet50, feature_module=model.resnet50.layer4, target_layer_names=["2"], use_cuda=True)

In [ ]:
_, dataset_val, _ = make_data()

In [ ]:
config

In [9]:
val_anomaly = dataset_val.with_annotation()
#dataset_val = val_anomaly
dataloader_val = DataLoader(dataset_val, num_workers=4, collate_fn=bbox_collate)
unnormalize = transf.UnNormalize(dataset_means['mean'], dataset_means['std'])
normalize = transf.Normalize(dataset_means['mean'], dataset_means['std'])

creating index...
index created!


In [10]:
class loss_r(nn.Module):
    
    def __init__(self):
        super(loss_r, self).__init__()
        
    def forward(self, i, target):
        loss = (target-i)**2
        loss = loss.sum()
        return 0.5*loss
    
class loss_l2(nn.Module):
    def __init__(self):
        super(loss_l2, self).__init__()
    
    def forward(self, i, rho = 1e-2):
        loss = i**2
        loss = loss.sum()
        return 0.5*rho*loss
    
class loss_tv(nn.Module):
    
    def __init__(self):
        super(loss_tv, self).__init__()
    
    def forward(self, i, target, rho = 1):
        w, h = i.shape[0], i.shape[1]
        lx = i[1:, :h-1] - target[:, :w-1, :h-1]
        ly = i[:w-1, 1:] - target[:, :w-1, :h-1]
        lx, ly = abs(lx), abs(ly)
        loss = lx.sum()+ly.sum()
        return rho*loss

In [11]:
class Loss(nn.Module):
    def __init__(self):
        super(Loss, self).__init__()
        self.l2loss = loss_l2()
        self.TVloss = loss_tv()
        self.r_loss = loss_r()
        
    def forward(self, i, t):
        #l2loss = loss_l2(i)
        #TVloss = loss_tv(i,t)
        #r_loss = loss_r(i,t)
       # s = time.time()
        l2loss = self.l2loss(i)
        TVloss = self.TVloss(i,t)
        r_loss = self.r_loss(i,t)
        #e = time.time()
        #print(f"calc_loss {e-s}")
        loss = l2loss + TVloss + r_loss
        return loss

In [12]:
def converge_map(init, mask):
    #init 512,512 ndarray ; mask b,512,512 ndarray
    seq1 = iaa.Sequential([
                    iaa.Affine(
        rotate=iap.DiscreteUniform(-180,170)*(-1)
                    )])
    ia.seed(0)
    mask_reconvert = torch.from_numpy(seq1(images=mask)) #b,512,512 tensor
    mp = torch.from_numpy(init).requires_grad_(True) #512,512 tensor
    #optimizer = optim.SGD([mp], lr = 1e-3)
    #optimizer = optim.LBFGS([mp])
    calc_loss = Loss()
    calc_loss.cuda()
    optimizer = optim.Adam([mp], lr = 5e-3)
    losses = []
    """
    for j in range(mask.shape[0]):
        plt.imshow(mask_reconvert[j])
        plt.show()
    """
    for i in range(200):
        optimizer.zero_grad()
        loss = calc_loss(mp.cuda(), mask_reconvert.cuda())
        #s = time.time()
        loss.backward()
        #e = time.time()
        #print(f"backward {e-s}")
        optimizer.step()
        #e2 = time.time()
        #print(f"optim {e2-e}")
        losses.append(loss)
    plt.figure()
    plt.plot(range(len(losses)), losses, linestyle = '-', color = 'red', label = 'loss')
    plt.xlabel('times')
    plt.ylabel('loss')
    plt.legend()
    plt.show()
    mp = mp.data.numpy()
    return mp #512,512 ndarray

In [13]:
def augmented_grad_cam(gcam, img):           #gcamはheatmapとlabelを出力するクラス
    #img.shape=B,C,H,W　tensor
    img = img.squeeze().numpy().transpose(1,2,0)  #512,512,3
    b = 10
    img = np.tile(img,(b,1,1,1)) #b,512,512,3
    seq = iaa.Sequential([
                    iaa.Affine(
        rotate=iap.DiscreteUniform(-180, 179)
                    )])
    ia.seed(0)
    img = seq(images=img)
    img = torch.from_numpy(img)
    labels, masks = grad_cam(img.permute(0,3,1,2), None) #label [1,0,0,0] masks [(40,512,512), 0, (40,512,512), 0]
    maps = []
    for i, label in enumerate(labels):
        if label == 0:
            maps.append(0)
            continue
        elif i == len(labels)-1:
            continue
        else:
            init_map = masks[i][0] #512,512
            mp = converge_map(init_map, masks[i])
            #plt.imshow(mp)
            #plt.show()
            #print(mp)
            maps.append(mp[np.newaxis,:,:])
    
    return maps
    

In [14]:
def heatmap2box(heatmap, img=0, threshold = 0.5):
    # img 512,512,3 ndarray,      heatmap  1,512,512 ndarray
    if not isinstance(img, numbers.Number):
        image = img.copy()
    heatmap = thresh(heatmap, threshold = threshold)
    heatmap = heatmap[0]
    heatmap = np.uint8(255*heatmap)
    label = cv2.connectedComponentsWithStats(heatmap)
    n = label[0] - 1
    data = np.delete(label[2], 0, 0)
    boxes = torch.tensor([])
    for i in range(n):
    # 各オブジェクトの外接矩形を赤枠で表示
        x0 = data[i][0]
        y0 = data[i][1]
        x1 = data[i][0] + data[i][2]
        y1 = data[i][1] + data[i][3]
        if boxes.shape[0] == 0:
            boxes = torch.tensor([[x0,y0,x1,y1]])
        else:
            torch.cat((boxes, torch.tensor([[x0,y0,x1,y1]])), dim=0)
        score = threshold
        if not isinstance(img, numbers.Number):
            cv2.rectangle(image, (x0, y0), (x1, y1), (255, 255, 0), thickness = 4)
    #if not isinstance(img, numbers.Number):
    #    plt.imshow(image)
    #    plt.show()
    return boxes, score

In [16]:
#通常の評価　マスクなし

def draw_caption(image, box, caption):
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
threshold= .5
gt, tpa, fpa, tna = np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)
for idx, data in enumerate(dataloader_val): 
    for i in range(len(data["bboxes"][0])):
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        #print(label_name)
    
    masks = grad_cam(data["img"], None)
    target = np.zeros(3)
    if len(data["bboxes"][0]) != 0:
        target[int(data["labels"][0][0])] = 1
    output = masks[0]
    
    
    gt += target
    tp = (output * target)
    fp = (output * (1 - target))
    tn = np.all((1 - output) * (1 - target))
    tpa += tp
    fpa += fp
    tna += tn
    #break
    #if isinstance(masks[2],numbers.Number):
    #        continue
    #masksa = augmented_grad_cam(grad_cam, data["img"]) 
    masks = masks[1]
    data['img'] = data['img'].squeeze(dim=0).numpy()
    img = unnormalize(data)['img'].copy() 
    img[img<0] = 0
    img[img>255] = 255
    flag = 0
    for num, mask in enumerate(masks):
        if isinstance(mask,numbers.Number):
            flag += 1
            """if flag == 3:
                print("Normal")"""
            continue
        """print("----------------------------------------")
        print("Grad-CAM")
        if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")"""
        #mask 1,512,512
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad1 = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad1, threshold = threshold)
        #plt.imshow(grad1)
        #plt.show()
        grad1 = cv2.cvtColor(grad1, cv2.COLOR_BGR2RGB)
        #cv2.imwrite(f"/data/unagi0/masaoka/w_o_mask_grad_cam/{idx}_{num}.png", np.uint8(255*grad1))
    #print("------------------------------------")
    """print("Augmented Grad-CAM")
    for num, mask in enumerate(masksa):
        if isinstance(mask,numbers.Number):
            continue
        if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")
        else:
            print("Normal")
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad, threshold = threshold)
        plt.imshow(grad)
        plt.show()"""
    #print("-------------------------------------")
    #print("Ground Truth")
    #plt.imshow(img)
    #plt.show()
    label = 3
    for i in range(len(data["bboxes"][0])):
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        draw_caption(img, (x1, y1, x2, y2), label_name)
        if label_name == "ulcer":
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2) #緑
        else:
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2) #赤
        label = int(data["labels"][0][i])
        #print(label_name)
        
    #plt.imshow(img)
    #plt.show()
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #cv2.imwrite(f"/data/unagi0/masaoka/w_o_mask_normal/{idx}_{label}.png", img)
    
print(tpa,gt-tpa,fpa,tna)

[1099.   77.   75.] [2. 0. 0.] [12.  0.  1.] [7770. 7770. 7770.]


In [15]:
#腫瘍にマスク、正常画像はランダムにマスク
def draw_caption(image, box, caption):
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
threshold= .5
gt, tpa, fpa, tna = np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)
all_masks = []
size = config["inputsize"]
for idx, data in enumerate(dataloader_val): 
    mask = torch.ones((size,size))
    flag = -1
    for i in range(len(data["bboxes"][0])):
        flag = 1
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        mask[y1:y2,x1:x2] = 0
        inv_mask = 1 - mask
        if (y2-y1)*(x2-x1) < size*size/4:
            all_masks.append(mask)
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        #print(label_name)
    if flag != 1:
        mask = random.choice(all_masks)
        inv_mask = 1 - mask
        
    blur = gaussian_filter(data["img"]*inv_mask,10)
    image = data["img"]*mask
    data["img"] = image + blur
    masks = grad_cam(data["img"], None)
    target = np.zeros(3)
    if len(data["bboxes"][0]) != 0:
        target[int(data["labels"][0][0])] = 1
    output = masks[0]
    
    
    gt += target
    tp = (output * target)
    fp = (output * (1 - target))
    tn = np.all((1 - output) * (1 - target))
    tpa += tp
    fpa += fp
    tna += tn
    #break
    #if isinstance(masks[2],numbers.Number):
    #        continue
    #masksa = augmented_grad_cam(grad_cam, data["img"]) 
    masks = masks[1]
    data['img'] = data['img'].squeeze(dim=0).numpy()
    img = unnormalize(data)['img'].copy() 
    img[img<0] = 0
    img[img>255] = 255
    flag = 0
    for num, mask in enumerate(masks):
        if isinstance(mask,numbers.Number):
            flag += 1
            """if flag == 3:
                print("Normal")"""
            continue
        #print("----------------------------------------")
        #print("Grad-CAM")
        """if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")"""
        #mask 1,512,512
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad1 = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad1, threshold = threshold)
        grad1 = cv2.cvtColor(grad1, cv2.COLOR_BGR2RGB)
        #cv2.imwrite(f"/data/unagi0/masaoka/w_mask_grad_cam/{idx}_{num}.png", np.uint8(255*grad1))
    #print("------------------------------------")
    """print("Augmented Grad-CAM")
    for num, mask in enumerate(masksa):
        if isinstance(mask,numbers.Number):
            continue
        if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")
        else:
            print("Normal")
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad, threshold = threshold)
        plt.imshow(grad)
        plt.show()"""
    #print("-------------------------------------")
    #print("Ground Truth")
    #plt.imshow(img)
    #plt.show()
    label = 3
    for i in range(len(data["bboxes"][0])):
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        draw_caption(img, (x1, y1, x2, y2), label_name)
        if label_name == "ulcer":
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2) #緑
        else:
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2) #赤
        label = int(data["labels"][0][i])
        
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #cv2.imwrite(f"/data/unagi0/masaoka/w_mask_normal/{idx}_{label}.png", img)
    
print(tpa,gt-tpa,fpa,tna)

(array([1087.,   19.,   72.]),
 array([14., 58.,  3.]),
 array([504.,  26.,  12.]),
 array([7296., 7296., 7296.]))

In [17]:
# 学習データ　マスクなし TP(異常を異常と判断)[3752.  444.  322.] FN(異常を正常と判断)[18.  0.  0.] 
#FP(正常を異常と判断)[24. 34.  2.] TN(正常を正常と判断)[27011. 27011. 27011.]

In [18]:
# 学習データ　マスクあり　　TP(異常部を削除して異常と判断)[2740.  182.  317.]  　FN(異常部を削除して正常と判断)[1030.  262.    5.]　
# FP(真に正常を異常と判断)[488.  88. 260.] 　TN(真に正常を正常と判断)[26620. 26620. 26620.]

In [19]:
#異常部分のみ切り出し　(異常部を見て異常と判断)[3572.  426.  269.] (異常部を見て正常と判断)[198.  18.  53.] 
#(正常切り出しを異常と判断)[1672.  299.  496.]　(正常切り出しを正常と判断)[24742. 24742. 24742.]

In [ ]:
_, dataset_val, _ = make_data()
val_anomaly = dataset_val.with_annotation()
#dataset_val = val_anomaly
dataloader_val = DataLoader(dataset_val, num_workers=3, collate_fn=bbox_collate)
unnormalize = transf.UnNormalize(dataset_means['mean'], dataset_means['std'])
normalize = transf.Normalize(dataset_means['mean'], dataset_means['std'])

In [ ]:
#腫瘍を切り出し表示、正常画像はランダムに切り出し表示
import random
def draw_caption(image, box, caption):
    b = np.array(box).astype(int)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 0), 2)
    cv2.putText(image, caption, (b[0], b[1] - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 255, 255), 1)
threshold= .5
gt, tpa, fpa, tna = np.zeros(3),np.zeros(3),np.zeros(3),np.zeros(3)
all_masks = []
for idx, data in enumerate(dataloader_val): 
    mask = torch.zeros((256,256))
    flag = 0
    for i in range(len(data["bboxes"][0])):
        flag = 1
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        mask[y1:y2,x1:x2] = 1
        inv_mask = 1 - mask
        if (y2-y1)*(x2-x1) < size*size/4:
            all_masks.append(mask) 
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        #print(label_name)
    if flag ==0:
        mask = random.choice(all_masks)
        inv_mask = 1 - mask
    blur = gaussian_filter(data["img"]*inv_mask,10)
    image = data["img"]*mask
    data["img"] = image + blur
    masks = grad_cam(data["img"], None)
    target = np.zeros(3)
    if len(data["bboxes"][0]) != 0:
        target[int(data["labels"][0][0])] = 1
    output = masks[0]
    
    
    gt += target
    tp = (output * target)
    fp = (output * (1 - target))
    tn = np.all((1 - output) * (1 - target))
    tpa += tp
    fpa += fp
    tna += tn
    #break
    #if isinstance(masks[2],numbers.Number):
    #        continue
    #masksa = augmented_grad_cam(grad_cam, data["img"]) 
    masks = masks[1]
    data['img'] = data['img'].squeeze(dim=0).numpy()
    img = unnormalize(data)['img'].copy() 
    img[img<0] = 0
    img[img>255] = 255
    flag = 0
    for num, mask in enumerate(masks):
        if isinstance(mask,numbers.Number):
            flag += 1
            """if flag == 3:
                print("Normal")"""
            continue
        """print("----------------------------------------")
        print("Grad-CAM")
        if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")"""
        #mask 1,512,512
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad1 = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad1, threshold = threshold)
        grad1 = cv2.cvtColor(grad1, cv2.COLOR_BGR2RGB)
        #cv2.imwrite(f"/data/unagi0/masaoka/tumor_only_grad_cam/{idx}_{num}.png", np.uint8(255*grad1))
    #print("------------------------------------")
    """print("Augmented Grad-CAM")
    for num, mask in enumerate(masksa):
        if isinstance(mask,numbers.Number):
            continue
        if num == 0:
            print("torose lesion")
        elif num == 1:
            print("vascular lesion")
        elif num == 2:
            print("ulcer")
        else:
            print("Normal")
        #heatmap2box(mask, img=img, threshold = threshold)
        mask_thresh = thresh(mask, threshold = threshold)
        grad = cam_on_image(img, mask_thresh[0])
        heatmap2box(mask, img=grad, threshold = threshold)
        plt.imshow(grad)
        plt.show()"""
    #print("-------------------------------------")
    #print("Ground Truth")
    #plt.imshow(img)
    #plt.show()
    label = 3
    for i in range(len(data["bboxes"][0])):
        x1 = int(data["bboxes"][0][i][0])
        y1 = int(data["bboxes"][0][i][1])
        x2 = int(data["bboxes"][0][i][2])
        y2 = int(data["bboxes"][0][i][3])
        label_name = dataset_val.labels[int(data["labels"][0][i])]
        draw_caption(img, (x1, y1, x2, y2), label_name)
        if label_name == "ulcer":
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=2) #緑
        else:
            cv2.rectangle(img, (x1, y1), (x2, y2), color=(255, 0, 0), thickness=2) #赤
        label = int(data["labels"][0][i])
        
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #cv2.imwrite(f"/data/unagi0/masaoka/tumor_only_normal/{idx}_{label}.png", img)
    
print(tpa,gt-tpa,fpa,tna)

In [21]:
# valid data [799.  70.  75.] [21. 24.  1.] [188. 334.  72.] [6979. 6979. 6979.]

In [22]:
# valid data 異常部を削除、正常画像はランダムに削除　[522.   2.  59.] [298.  92.  17.] [911. 381.  23.] [6213. 6213. 6213.]

In [23]:
# valid data 異常画像のみ切り出し、正常画像はランダムに切り出し　[791.  83.  32.] [29. 11. 44.] [1392.  310.   11.] [5705. 5705. 5705.]

In [ ]:
# 学習データ　マスクなし TP(異常を異常と判断)[799.  70.  75.]  FN(異常を正常と判断)[21. 24.  1.]
#FP(正常を異常と判断)[188. 334.  72.]  TN(正常を正常と判断)[6979. 6979. 6979.]

In [ ]:
# 学習データ　マスクあり　　TP(異常部を削除して異常と判断)[522.   2.  59.] 　FN(異常部を削除して正常と判断)[298.  92.  17.]　
# FP(真に正常を異常と判断) [911. 381.  23.]　TN(真に正常を正常と判断)[6213. 6213. 6213.]

In [ ]:
#異常部分のみ切り出し　(異常部を見て異常と判断)[791.  83.  32.]  (異常部を見て正常と判断)[29. 11. 44.] 
#(正常切り出しを異常と判断)[1392.  310.   11.] 　(正常切り出しを正常と判断)[5705. 5705. 5705.]

In [24]:
tp = np.array([799. , 70. , 75.])
fn = np.array([21. ,24.  ,1.] )
fp = np.array([188., 334. , 72.])
tn = np.array([6979. ,6979. ,6979.])
recall = tp/(tp+fn)
s = tn/(tn+fp)
print(recall, s)
#精度はバランスが取れてとても良い

[0.97439024 0.74468085 0.98684211] [0.97376866 0.95432791 0.98978868]


In [25]:
tp = np.array([523.   ,3.  ,65.])
fn = np.array([297.  ,91.  ,11.] )
fp = np.array([911., 381. , 23.])
tn = np.array([6213., 6213., 6213.])
recall = tp/(tp+fn)
s = tn/(tn+fp)
print(recall, s)
#異常部分を消した元異常画像が誤って異常と検出

[0.63780488 0.03191489 0.85526316] [0.8721224  0.9422202  0.99631174]


In [26]:
tp = np.array([762.,  72. , 72.])
fn = np.array([58. ,22.  ,4.] )
fp = np.array([ 12.,  57. ,180.])
#tn += np.array([6979. ,6979. ,6979.])
recall = tp/(tp+fn)
print(recall)
#異常部分だけ切り出して検出

[0.92926829 0.76595745 0.94736842]


In [29]:
dataset_val.load_annotations(1)

{'annot': array([[284., 330., 470., 485.,   0.]]),
 'bboxes': array([[284., 330., 470., 485.]], dtype=float32),
 'labels': array([0])}